In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Series de Tiempo - Checkpoint

<br/> 

<div id="caja11" style="float:left;width: 100%;">
  <div style="float:left;width: 9%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;">
      <label>Vamos a poner en práctica lo aprendido en la notebook de práctica guiada.</label>
  <div style="float:left;width: 85%;">
      <label>Es importante que antes de la clase resuelvan esta notebook ya que es fundamental que sepan utilizar las herramientas que vimos en la práctica guiada para después trabajar el caso práctico en la clase.</label>        
</div>    
</div>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

from scipy import stats
from statistics import mode

from sklearn.model_selection import train_test_split

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

import warnings
warnings.filterwarnings('ignore')

from checkpoint_series_tiempo import *

#### Pongamos en práctica las herramientas que vimos en la práctica guiada: 

En este caso, nuestra variable de interés el el precio de la cebolla en India. Contamos con un dataset con información desagregada por ciudad. 

In [ ]:
df = pd.read_csv('../Data/cebolla_india.csv')
df.shape

In [ ]:
df.head()

El dataset tiene tres columnas (features o variables) sobre la ubicación del mercado mayorista donde se vendió cebolla:

* **state**: esta es la abreviatura de 2/3 letras para el estado en India (PB es Punjab, etc.)
* **ciudad**: esta es la ciudad en India (ABOHAR, BANGALORE y así sucesivamente)
* **mercado**: esta es una cadena con la combinación del estado y la ciudad

Tres están relacionados con la fecha de la transacción: 

* **mes**: mes en enero, febrero y así sucesivamente
* **año**: año (YYYY)
* **fecha**: la combinación de los dos anteriores

Cuatro son acerca de la cantidad y el precio en este mercado mayorista (todas numéricas).

* **cantidad**: la cantidad de cebolla que llega al mercado en ese mes en quintales (100 kg)
* **priceMin**: el precio mínimo en el mes en Rs./quintal
* **priceMax**: el precio máximo en el mes en Rs./quintal
* **priceMod**: el precio modal en el mes en Rs./quintal

Fijate de qué tipo son las columnas del DataFrame:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df.dtypes

Podés observar que "date" es un object. Transformala a un datetime:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:

df['date'] = pd.to_datetime(df['date'])

df['date'].dtype

Vamos a analizar el volumen de ventas por ciudad. Hacé un groupby por ciudad y sumá las cantidades vendidas ('quantity'):

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df.groupby('city')['quantity'].sum().sort_values(ascending=False).head()

Vamos a restringir nuestro análisis solamente a la ciudad de Bangalore. Generá un DataFrame nuevo conservando solamente a la serie de tiempo de Bangalore:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
dfBang = df.loc[df.city == 'BANGALORE'].copy()

Hacé un sort del DataFrame para que las observaciones queden ordenadas en el tiempo:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
dfBang = dfBang.sort_values(by = "date")

Reindexá el DataFrame para que el index sea "date" con frecuencia mensual:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
dfBang.index = pd.PeriodIndex(dfBang.date, freq='M')
dfBang.head()

Vamos a definir una función que plotea series de tiempo:

In [ ]:
# Función que plotea la serie:
def plot_df(df, x, y, title="", xlabel='Fecha', ylabel='Valor', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

Vamos a trabajar usando la serie **priceMod**.

Usá la función que acabamos de definir para plotear 'priceMod'. Recordá que las x tienen que tener un formato de datetime. 

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
plot_df(dfBang, x=dfBang.date, y=dfBang.priceMod, title="priceMod")

Ahora creá una dummy de tiempo para modelar la tendencia lineal:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:

dfBang["timeIndex"] = pd.Series(np.arange(len(dfBang['priceMod'])), index=dfBang.index)

dfBang.head()

# Comentario: recordá que la dummy de tiempo es una variable de apoyo que lleva la cuenta de
# la cantidad de periodos que pasan desde la primera observación. Sirve para poder hacer
# regresiones contra el tiempo y estimar la tendencia. 

In [ ]:
dfBang.tail()

Ahora creá las dummies de mes y hacé el join con el DataFrame.

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
dummies_mes = pd.get_dummies(dfBang['month'], drop_first=True)
dfBang =  dfBang.join(dummies_mes)
dfBang.sample(10)

Hacé el split entre train y test dejando los últimos 12 meses en el set de testeo. Luego corroborá la continuidad entre el set de entrenamiento y de testeo. 

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train, df_test = train_test_split(dfBang, test_size=12, random_state=42, shuffle=False)

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train.tail()

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.head()

Creá las transformaciones logarítmicas de priceMod tanto para el set de entrenamiento como para el set de testeo.

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train['log_value'] = np.log(df_train['priceMod'])
df_test['log_value'] = np.log(df_test['priceMod'])

Ploteá la serie logarítmica:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
plot_df(df_train, x=df_train.date, y=df_train['log_value'],\
    title='Log de priceMod del train set')

Vemos que la dispersión de la serie se estabilizó significativamente en t. 

Ahora entrená un modelo lineal entre la serie transformada y la dummy de tiempo y analizá el summary.

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
model_log = smf.ols('log_value ~ timeIndex',\
                          data = df_train).fit()

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
model_log.summary()

Agregá las predicciones del modelo en el set de entrenamiento y de testeo con y sin back-transformation:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:

df_train['model_log'] = model_log.predict(df_train[["timeIndex"]])
df_test['model_log'] = model_log.predict(df_test[["timeIndex"]])

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:

df_train['back_model_log'] = np.exp(df_train['model_log'])
df_test['back_model_log'] = np.exp(df_test['model_log'])

Ploteá las predicciones vs. las series reales, tanto en el set de entrenamiento como en el de testeo.

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train.plot(kind = "line", x = "date", y = ['log_value', 'model_log']);

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train.plot(kind = "line", x = "date", y = ['priceMod', 'back_model_log']);

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", x = "date", y = ['log_value', 'model_log']);

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", x = "date", y = ['priceMod', 'back_model_log']);

Creamos la función para calcular el RMSE:

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

Guardá el resultado en un DataFrame:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_Results = pd.DataFrame(columns = ["Model", "RMSE"])
df_Results.loc[0, "Model"] = "Log"
df_Results.loc[0, "RMSE"] = RMSE(df_test.back_model_log, df_test.priceMod)
df_Results

Ahora entrená un modelo agregando variables de estacionalidad mensual y agregá el RMSE en el DataFrame de resultados. 

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:

model_log_est = smf.ols('log_value ~ timeIndex + August + December + February + January + July + November+ October + September',\
                          data = df_train).fit()

# Recordá que tenés que agregar a la función de regresión los nombres de las dummies
# mensuales que agregamos antes. Recordá también que dropeamos un mes, por lo cual no lo 
# tenés que incluir en la expresión de la regresión. 
# En la solución no estamos incluyendo a los meses de junio, marzo y mayo porque no
# son significativos.



In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:

model_log_est.summary()

Comentario: recordá que podés usar el método precict del modelo para realizar predicciones.
Al método le tenés que pasar el DataFrame y especificar las columnas a incluir. 

Hacé las predicciones en el set de entrenamiento y testo y almacená los resultados en ambos DataFrames:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:



df_train['model_log_est'] = model_log_est.predict(df_train[["timeIndex",\
                                              "August", "December", "February", "January",\
                                               "July","November", "October", "September"]])


df_test['model_log_est'] = model_log_est.predict(df_test[["timeIndex",\
                                              "August", "December", "February", "January",\
                                               "July","November", "October", "September"]])

Comentario: recordá que para hacer back transformation de una transformación logarítmica tenés que usar la función exponencial. 

Almacená en tus DataFrames los modelos con back transformation. 

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train['back_model_log_est'] = np.exp(df_train['model_log_est'])
df_test['back_model_log_est'] = np.exp(df_test['model_log_est'])

Plotea el modelo con y sin back transformation para el set de entrenamiento:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train.plot(kind = "line", x = "date", y = ['log_value', 'model_log_est']);

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train.plot(kind = "line", x = "date", y = ['priceMod', 'back_model_log_est']);

Plotea el modelo con y sin back transformation para el set de testeo:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", x = "date", y = ['log_value', 'model_log_est']);

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", x = "date", y = ['priceMod', 'back_model_log_est']);

Calculá el RMSE del modelo con transformación logarítmica y estacionalidad mensual y agregala al DataFrame de resultados: 

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_Results.loc[1, "Model"] = "Log + Est"
df_Results.loc[1, "RMSE"] = RMSE(df_test.back_model_log_est, df_test.priceMod)
df_Results